In [1]:
# imports

from keras.applications import InceptionV3, ResNet50 # vgg16
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input

from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

Using TensorFlow backend.


In [10]:
# parameters setup
#imgs_path = "M:/VieApp/impressionim_bio/"
#imgs_path = "C:/Users/CON3ADOLEA/Downloads/wikiart/wikiart/Impressionism/"
imgs_path = "M:/VieApp/images/images/"
imgs_model_width, imgs_model_height = 224, 224

nb_closest_images = 5 # number of most similar images to retrieve

In [11]:
# load the model
#vgg_model = vgg16.VGG16(weights='imagenet')
#vgg_model = InceptionV3.InceptionV3(weights='imagenet')
vgg_model = ResNet50(weights='imagenet')


# remove the last layers in order to get features instead of predictions
#feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc1000").output)



# print the layers of the CNN
#feat_extractor.summary()

In [12]:
files = [imgs_path + x for x in os.listdir(imgs_path) if "jpg" in x]

print("number of images:",len(files))

number of images: 4895


In [13]:
# load all the images and prepare them for feeding into the CNN

importedImages = []

for f in files:
    filename = f
    original = load_img(filename, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    
    importedImages.append(image_batch)
    
images = np.vstack(importedImages)

processed_imgs = preprocess_input(images.copy())

In [14]:
# extract the images features

imgs_features = feat_extractor.predict(processed_imgs)
###save the img feature
np.save('images_feature.npy',imgs_features)

print("features successfully extracted!")
imgs_features.shape

features successfully extracted!


(4895, 1000)

In [19]:
# compute cosine similarities between images
imgs_features=np.load('images_feature.npy')
cosSimilarities = cosine_similarity(imgs_features)

# store the results into a pandas dataframe

cos_similarities_df = pd.DataFrame(cosSimilarities, columns=files, index=files)
#cos_similarities_df.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 251


In [46]:
cos_similarities_df.head()

,M:/VieApp/images/images/11pmfromtheseriesTwen.jpg,M:/VieApp/images/images/12NummiCoinofaByzantine.jpg,M:/VieApp/images/images/17Fragments18thcentury.jpg,M:/VieApp/images/images/18601870publishedJanuary8.jpg,M:/VieApp/images/images/1ArchofTitus2VillaFar.jpg,M:/VieApp/images/images/A10MassachusettsDiningRoom.jpg,M:/VieApp/images/images/A11RhodeIslandParlorc1.jpg,M:/VieApp/images/images/A12CapeCodLivingRoom175.jpg,M:/VieApp/images/images/A13NewEnglandBedroom1750.jpg,M:/VieApp/images/images/A14PennsylvaniaDrawingRoom.jpg,...,M:/VieApp/images/images/OntheBankoftheSeineatPo.jpg,M:/VieApp/images/images/OntheBankoftheSeineBenn.jpg,M:/VieApp/images/images/OntheBeachTwoAreCompany.jpg,M:/VieApp/images/images/OntheRoad1856.jpg,M:/VieApp/images/images/OntheSeinec1895.jpg,M:/VieApp/images/images/OntheStage187677.jpg,M:/VieApp/images/images/OntheStageplateonefromY.jpg,M:/VieApp/images/images/OntheTerraceofaHotelinB.jpg,M:/VieApp/images/images/OntheTestc1859.jpg,M:/VieApp/images/images/OntheWavesofLove1896.jpg
M:/VieApp/images/images/11pmfromtheseriesTwen.jpg,1.000000,0.024582,0.345513,0.992809,0.002604,0.000506,0.000386,0.000763,0.000250,0.000234,...,0.003432,0.015611,0.655870,0.016740,0.001207,0.008252,0.546218,0.986872,0.052988,0.748998
M:/VieApp/images/images/12NummiCoinofaByzantine.jpg,0.024582,1.000000,0.007331,0.008344,0.001867,0.001535,0.001950,0.005300,0.002419,0.002957,...,0.003525,0.003642,0.013491,0.000926,0.001764,0.003040,0.128949,0.005314,0.003856,0.137010
M:/VieApp/images/images/17Fragments18thcentury.jpg,0.345513,0.007331,1.000000,0.347016,0.003703,0.000119,0.000617,0.000405,0.000223,0.000124,...,0.003008,0.062064,0.230023,0.005814,0.000601,0.005546,0.197427,0.344889,0.025295,0.255478
M:/VieApp/images/images/18601870publishedJanuary8.jpg,0.992809,0.008344,0.347016,1.000000,0.004146,0.001638,0.001647,0.003391,0.000733,0.002151,...,0.004781,0.015572,0.662384,0.016815,0.001256,0.008294,0.498520,0.997000,0.055837,0.696083
M:/VieApp/images/images/1ArchofTitus2VillaFar.jpg,0.002604,0.001867,0.003703,0.004146,1.000000,0.008312,0.041196,0.019101,0.014660,0.007936,...,0.138566,0.017461,0.055709,0.001786,0.005836,0.002902,0.004100,0.002829,0.001704,0.002951


In [48]:
# function to retrieve the most similar products for a given one
import os
import shutil
def retrieve_most_similar_products(given_img):

    print("--"*5)
    print("original product:")

    original = load_img(given_img, target_size=(imgs_model_width, imgs_model_height))
    plt.imshow(original)
    plt.show()

    print("-----------------------------------------------------------------------")
    print("most similar products:")

    #closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1].index
    #closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1]
    #print(cos_similarities_df[given_img].sort_values(ascending=False))

    closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+26].index
    #print(closest_imgs)
    closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+26]
   
    destination_folder = 'C:/Users/CON3ADOLEA/Downloads/GNN-RecSys-main/recommenders-main/bio_dataset/'
    

    for i in range(0,len(closest_imgs)):
        original = load_img(closest_imgs[i], target_size=(imgs_model_width, imgs_model_height))
        plt.imshow(original)
        plt.show()
        print(closest_imgs[i])
        if closest_imgs[i]:
            shutil.copy(closest_imgs[i],destination_folder) ## use to copy files into a folder
                      
        print("similarity score : ",closest_imgs_scores[i])

In [50]:
#retrieve_most_similar_products(files[0])
#files[1]
#retrieve_most_similar_products('M:/VieApp/impressionim_bio/abdullah-suriosubroto_air-terjun.jpg')
#retrieve_most_similar_products("M:/VieApp/images/1ArchofTitus2VillaFar.jpg")

In [25]:
cos_similarities_df.to_csv('feature_similarities.csv', encoding='utf-8')

In [ ]:
def retrieve_most_similar_products(given_img):

    closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+26].index
    #print(closest_imgs)
    closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+26]
   
    destination_folder = 'C:/Users/CON3ADOLEA/Downloads/GNN-RecSys-main/recommenders-main/bio_dataset/'
    

    for i in range(0,len(closest_imgs)):

        if closest_imgs[i]:
            shutil.copy(closest_imgs[i],destination_folder) ## use to copy files into a folder
                      
        print("similarity score : ",closest_imgs_scores[i])

In [39]:
#from scipy import sparse
cos_similarities_df['M:/VieApp/images/images/11pmfromtheseriesTwen.jpg'].sort_values(ascending=False)[1:nb_closest_images]
#cos_similarities_df.head()
#closest_imgs = cos_similarities_df.sort_values(ascending=False)[1:nb_closest_images+3].index
#print(closest_imgs)
#closest_imgs_scores = cos_similarities_df.sort_values(ascending=False)[1:nb_closest_images+3]
   
#destination_folder = 'C:/Users/CON3ADOLEA/Downloads/GNN-RecSys-main/recommenders-main/bio_dataset/'
  

#for i in range(0,len(closest_imgs)):

#    if closest_imgs[i]:
#        shutil.copy(closest_imgs[i],destination_folder) ## use to copy files into a folder
                      
#print("similarity score : ",closest_imgs_scores[i])

M:/VieApp/images/images/AkashioftheTamayafromthe.jpg    0.999355
M:/VieApp/images/images/BuddhafromtheSuiteofLate.jpg    0.998916
M:/VieApp/images/images/HouroftheSheep2pmhits.jpg       0.998204
M:/VieApp/images/images/HenjofromtheseriesSixIm.jpg     0.997604
Name: M:/VieApp/images/images/11pmfromtheseriesTwen.jpg, dtype: float32

In [41]:
cos_similarities_df.columns.sort_values(ascending=False)[1:nb_closest_images]

Index(['M:/VieApp/images/images/OntheTestc1859.jpg',
       'M:/VieApp/images/images/OntheTerraceofaHotelinB.jpg',
       'M:/VieApp/images/images/OntheStageplateonefromY.jpg',
       'M:/VieApp/images/images/OntheStage187677.jpg'],
      dtype='object')

In [18]:
#filenames = "C:/Users/CON3ADOLEA/Downloads/wikiart/wikiart/Impressionism/zinaida-serebriakova_autumn-landscape-1904.jpg"
#input_img = load_img(filenames, target_size=(224, 224))
#numpy_image = img_to_array(input_img)
#image_batch = np.expand_dims(numpy_image, axis=0)
#processed_imgs = preprocess_input(images.copy())                     
#retrieve_most_similar_products()

from PIL import Image
from numpy import asarray
# load the image
image = Image.open('C:/Users/CON3ADOLEA/Downloads/wikiart/wikiart/Impressionism/zinaida-serebriakova_autumn-landscape-1904.jpg')
# convert image to numpy array
data = asarray(image)
print(type(data))
# summarize shape
print(data.shape)


<class 'numpy.ndarray'>
(1000, 819, 3)


In [43]:
for i in cos_similarities_df.columns:
    dcf = pd.DataFrame(cos_similarities_df[i].sort_values(ascending=False)[1:nb_closest_images].index)
    dc = dcf.rename({0:i},axis=1) # rename column names
    writer = pd.ExcelWriter('output.xlsx')

    d = pd.DataFrame(dc)
    
    d.to_excel(writer)
    writer.save()

ERROR! Session/line number was not unique in database. History logging moved to new session 286
